In [32]:
import random 
import numpy as np
import torch


In [33]:
def Loss(Y,beta,X,lamd,p,K):
    sum=0
    
    for k in range(K):
        sum_betaX=0

        for j in range(p):
            sum_betaX=sum_betaX+beta[j,k]*X[j,k,:]

        sum=sum+0.5*torch.norm(Y[k,:].flatten()-sum_betaX.flatten())**2
    
    norm_infinite,indice=torch.max(torch.abs(beta),1)
    sum=sum+lamd*torch.sum(norm_infinite)

    return sum


In [34]:
#initialize settings
p=10
K=5
n_k=60
lamd=1

X=torch.randn(p,K,n_k)
for j in range(p):
    for k in range(K):
        X[j,k,:]=X[j,k,:]/torch.norm(X[j,k,:])

beta_true=torch.randint(low=-5,high=5,size=[p,K])
# print(beta_true)
Y=torch.zeros(K,n_k)

for k in range(K):
    sum=0
    for j in range(p):
        sum=sum+beta_true[j,k]*X[j,k,:]
    Y[k,:]=sum+torch.tensor(np.random.normal(loc=0,scale=0,size=X[j,k,:].size())).float()

# print(Y.size())


In [35]:
#C_ij
C=torch.zeros(p,K)
for j in range(p):
    for k in range(K):
        C[j,k]=torch.matmul(X[j,k,:].flatten(),Y[k,:].flatten())
        # print(C[j,k])
#D_ij^k
D=torch.zeros(p,p,K)
for k in range(K):
    for i in range(p):
        for j in range(p):
            D[i,j,k]=torch.matmul(X[i,k,:].flatten(),X[j,k,:].flatten())


In [36]:
# beta=beta_true+torch.tensor(np.random.normal(loc=0,scale=9,size=beta_true.size())).float()
beta=torch.tensor(np.random.normal(loc=0,scale=25,size=beta_true.size())).float()
loss=Loss(Y,beta,X,lamd,p,K)
print(loss)

tensor(13209.6250)


In [37]:
def single_blcok(beta,p,K,lamd,X,Y,C,D,j_round):
    
    alpha=beta
    for k in range(K):
        beta[j_round,k]=C[j_round,k]-torch.matmul(beta[:,k].flatten().float(),D[:,j_round,k].flatten().float())+beta[j_round,k]*D[j_round,j_round,k]

    if (lamd!=0):
        if (torch.sum(torch.abs(alpha[j_round,:]))<=lamd):
            beta[j_round,:]=0
    
        alpha_j=alpha[j_round,:]
        alpha_j_abs=torch.abs(alpha_j)
        alpha_sorted,index1=torch.sort(alpha_j_abs,descending=True)
        alpha_j_sortred=torch.index_select(alpha_j,0,index1)
        judge=torch.zeros(size=alpha_j_abs.size())
    
        for l in range(K):
            judge[l]=(torch.sum(alpha_j_abs[0:l+1])-lamd)/(l+1)
    
        max_index=torch.argmax(judge)
    
    
        for k in range(max_index):
            beta[j_round,k]=torch.sign(alpha_j_sortred[k])/(max_index+1)*(torch.sum(alpha_j_abs[0:max_index+1])-lamd)

        for k in range(max_index,K):
            beta[j_round,k]=alpha_j_sortred[k]

        b,index2=torch.sort(index1)
    
        beta[j_round,:]=torch.index_select(beta[j_round,:],0,index2)
   
    return beta

In [38]:
delta_loss=100
while(delta_loss>=0.01):
    for j_round in range(p):
        beta=single_blcok(beta,p,K,lamd,X,Y,C,D,j_round)
    loss_new=Loss(Y,beta,X,lamd,p,K)
    delta_loss=torch.abs(loss_new-loss)
    loss=loss_new
    print(Loss(Y,beta,X,lamd,p,K))

print("%f",beta)
print("%f",beta_true)

tensor(769.2280)
tensor(136.0310)
tensor(67.8733)
tensor(53.7712)
tensor(48.3850)
tensor(47.6117)
tensor(46.5234)
tensor(46.6319)
tensor(46.6176)
tensor(46.6183)
%f tensor([[-4.0864,  4.3017,  3.2575,  3.7427, -0.0534],
        [-3.0852, -2.8753, -2.8753,  0.2821, -0.1163],
        [ 2.8477, -2.8477, -1.7008, -2.8477, -1.5585],
        [ 0.8479, -2.4942, -2.4942,  1.3078,  1.0989],
        [-5.1669,  2.0814, -1.2683,  3.9273, -3.0540],
        [-3.7731, -4.0670, -3.7191, -4.0670,  4.0670],
        [ 3.3890,  3.3890, -3.4010, -2.7718, -2.9910],
        [-4.9553,  2.8746, -2.9675,  0.3410, -0.9453],
        [ 2.5224,  2.9679, -1.5507,  1.6639, -2.6724],
        [-2.1463,  0.3278,  2.1463, -2.1463,  2.1463]])
%f tensor([[-5,  4,  3,  4,  0],
        [-3, -3, -3,  0,  0],
        [ 2, -4, -2, -5, -2],
        [ 1, -3, -5,  2,  1],
        [-5,  2, -1,  4, -3],
        [-4, -5, -3, -5,  4],
        [ 4,  4, -3, -3, -3],
        [-5,  3, -2,  0, -1],
        [ 3,  3, -2,  2, -3],
        [-1

In [39]:
loss=Loss(Y,beta,X,lamd,p,K)
print(loss)


tensor(46.6183)


In [40]:
lamd=lamd*0.5
delta_loss=100
while(delta_loss>=0.01):
    for j_round in range(p):
        beta=single_blcok(beta,p,K,lamd,X,Y,C,D,j_round)
    loss_new=Loss(Y,beta,X,lamd,p,K)
    delta_loss=torch.abs(loss_new-loss)
    loss=loss_new
    print(Loss(Y,beta,X,lamd,p,K))

tensor(28.1221)
tensor(28.0865)
tensor(28.1024)
tensor(28.1363)
tensor(28.1072)
tensor(28.1039)


In [41]:
lamd=lamd*0.5
delta_loss=100
while(delta_loss>=0.001):
    for j_round in range(p):
        beta=single_blcok(beta,p,K,lamd,X,Y,C,D,j_round)
    loss_new=Loss(Y,beta,X,lamd,p,K)
    delta_loss=torch.abs(loss_new-loss)
    loss=loss_new
    print(Loss(Y,beta,X,lamd,p,K))

tensor(18.4646)
tensor(18.4282)
tensor(18.4236)
tensor(18.4236)


In [42]:
lamd=lamd*0.5
delta_loss=100
while(delta_loss>=0.001):
    for j_round in range(p):
        beta=single_blcok(beta,p,K,lamd,X,Y,C,D,j_round)
    loss_new=Loss(Y,beta,X,lamd,p,K)
    delta_loss=torch.abs(loss_new-loss)
    loss=loss_new
    print(Loss(Y,beta,X,lamd,p,K))

tensor(13.5794)
tensor(13.5629)
tensor(13.5608)
tensor(13.5607)


In [43]:
print("%f",beta)
print("%f",beta_true)

%f tensor([[-4.8853,  4.2277,  3.2335,  3.7580, -0.0680],
        [-2.7347, -3.0122, -3.0122,  0.2501, -0.1452],
        [ 3.0834, -3.0834, -1.7113, -3.0834, -1.6001],
        [ 0.6321, -2.7181, -2.7181,  1.4280,  1.0756],
        [-5.2410,  1.9944, -1.2227,  3.9404, -3.0203],
        [-3.9901, -4.3473, -3.6446, -4.3473,  4.3473],
        [ 3.7869,  3.7377, -3.3761, -2.8060, -3.0325],
        [-5.0319,  2.8636, -2.8824,  0.3204, -0.9733],
        [ 2.8140,  2.8152, -1.5972,  1.7129, -2.6474],
        [-2.2606,  0.1865,  2.2606, -2.2606,  2.2606]])
%f tensor([[-5,  4,  3,  4,  0],
        [-3, -3, -3,  0,  0],
        [ 2, -4, -2, -5, -2],
        [ 1, -3, -5,  2,  1],
        [-5,  2, -1,  4, -3],
        [-4, -5, -3, -5,  4],
        [ 4,  4, -3, -3, -3],
        [-5,  3, -2,  0, -1],
        [ 3,  3, -2,  2, -3],
        [-1,  0,  4, -3,  4]])
